The purpose of this notebook is to carry out the calculation in Theorem 5.2 for the order number of the DAG $(G,W,\theta)$ defined at the beginning of Section 5.

In [99]:
"""
We will enter a DAG manually by specifying its incidence matrix. A "1" in row 
i / column j means that there is a directed edge from vertex j to vertex i.

The input "n" is the number of vertices in the graph, and the output "A" is the 
incidence matrix.
"""

def graph_entry(n):
  import numpy as np
  A = np.zeros((n, n), dtype = int)
  while True:
    citation = input("Enter row-column separated by space. Enter s to stop.")
    if citation == 's':
      break
    else:
      citation = [int(x) for x in citation.split(' ')]
      A[citation[0] - 1, citation[1] - 1] = 1
  return(A)

In [100]:
"""
An implementation of Algorithm 1 in Section 2. The input "A" is the incidence 
matrix of our DAG, and the output "count" is the number of acyclic orderings, 
i.e., the order number, of the DAG.  This is a very slow algorithm.
"""

def order_number(A):
  import numpy as np
  count = 0
  if np.shape(A)[0] == 1:
    count = 1
  else:
    for i in range(0, np.shape(A)[0]):
      if np.all(A[i] == 0):
        B = np.delete(A, i, axis = 0)
        C = np.delete(B, i, axis = 1)
        count = count + order_number(C)
  return count

In [ ]:
"""
We can now verify the order number of the DAG at the end of Section 2.
"""

G = graph_entry(15)
order_number(G)


Enter row-column separated by space. Enter s to stop.4 1
Enter row-column separated by space. Enter s to stop.5 1
Enter row-column separated by space. Enter s to stop.5 2
Enter row-column separated by space. Enter s to stop.6 3
Enter row-column separated by space. Enter s to stop.7 3
Enter row-column separated by space. Enter s to stop.8 3
Enter row-column separated by space. Enter s to stop.9 4
Enter row-column separated by space. Enter s to stop.9 1
Enter row-column separated by space. Enter s to stop.9 5
Enter row-column separated by space. Enter s to stop.10 2
Enter row-column separated by space. Enter s to stop.10 6
Enter row-column separated by space. Enter s to stop.11 7
Enter row-column separated by space. Enter s to stop.11 8
Enter row-column separated by space. Enter s to stop.12 8
Enter row-column separated by space. Enter s to stop.13 9
Enter row-column separated by space. Enter s to stop.14 9
Enter row-column separated by space. Enter s to stop.14 10
Enter row-column separ

8241100

In [101]:
"""
The order number of a tree can be computed directly using Theorem 4.2.  The 
following is an implementation of this theorem.
"""

"""
The input "T" of `branch_nums` is the incidence matrix of a tree.  The output 
"B" is matrix with a "1" in row i / column j if vertex i is a descendant of 
vertex j, and a "0" otherwise.  This function is used by `ord_num_tree` below.  
It makes use of Lemma 4.4.
"""

def branch_nums(T):
  import numpy as np
  from numpy.linalg import matrix_power
  n = len(T)
  B = np.zeros((n, n), dtype = int)
  i = 1
  while True:
    B = B + matrix_power(T, i)
    i += 1
    if np.all(matrix_power(T, i) == np.zeros((n, n))):
      break
  return(B)

"""
`ord_num_tree` takes the incidence matrix "T" of a tree as input and returns the 
tree's order number.  The output "B" of `branch_nums` is used to compute the 
branch numbers needed by Theorem 4.2.
"""

def ord_num_tree(T):
  import numpy as np
  from math import factorial
  n = len(T)
  S = T.copy()
  B = branch_nums(S)
  for i in range(0, n):
    for j in range(0, n):
      if S[j, i] == 1:
        S[j,i] = S[j,i] + np.sum(B[:, j])
  D = np.zeros((n, n), dtype = int)
  for k in range(0, n):
    for l in range(0, n):
      D[l, k] = factorial(S[l, k])
  count = 1
  for p in range(0, n):
    count = count * factorial(np.sum(S[:, p])) / np.product(D[:, p])
  return round(count) # `round` fixes the floating point error

In [ ]:
"""
We can now verify Example 4.3.
"""

T = graph_entry(10)
ord_num_tree(T)

Enter row-column separated by space. Enter s to stop.2 1
Enter row-column separated by space. Enter s to stop.3 1
Enter row-column separated by space. Enter s to stop.4 2
Enter row-column separated by space. Enter s to stop.5 2
Enter row-column separated by space. Enter s to stop.6 3
Enter row-column separated by space. Enter s to stop.7 3
Enter row-column separated by space. Enter s to stop.8 3
Enter row-column separated by space. Enter s to stop.9 7
Enter row-column separated by space. Enter s to stop.10 7
Enter row-column separated by space. Enter s to stop.s


6720

In [103]:
"""
To implement Theorem 5.2, we need a few preliminary functions.
"""

"""
`delete` performs an operation needed by `all_seqs`.  "x" and "a_pos_seqs" 
are lists, and "m" and "i" are positive integers.
"""

def delete(x, m, i, a_pos_seqs):
  if len(x) == m:
    A_list = x
    a_pos_seqs += [A_list]
  else:
    for j in range(i, len(x)):
      A_list = delete(x[0:j] + x[j + 1: len(x)], m, j, a_pos_seqs)
  return(a_pos_seqs)

  """
  `all_seqs` takes in non-negative integers "m" and "n" and returns 
  "ab_seqs", which is a list of the m + n choose n sequences of m 0's and 
  n 1's.  It is used by `formula` below.
  """

def all_seqs(m, n):
  ab_seqs = []
  a_seqs = delete(list(range(1, m + 1)), n, 0, [])
  for x in a_seqs:
    AB_list = [0]*m
    for i in range(1, m + 1):
      if i in x:
        AB_list[i - 1] = 0
      else:
        AB_list[i - 1] = 1
    ab_seqs += [AB_list]
  return(ab_seqs)

  """
  `all_seqs_2` accepts the output list from `all_seqs` as the input "seqs_list" 
  as well as "left" and "right", which are lists of lists of incidence 
  matrices of trees.  It returns a list of lists "H" of non-negative integers.  
  It is used by `formula` below to obtain the sequences of cardinalities of 
  the trees attached to the left and right branches of G.
  """

def all_seqs_2(seqs_list, left, right):
  import numpy as np
  n = len(left) + len(right)
  H = np.zeros((len(seqs_list), n), dtype = int)
  for seq in seqs_list:
    row = seqs_list.index(seq)
    lk = 0
    rk = 0
    for j in range(0, len(seq)):
      if seq[j] == 0:
        H[row, j] = sum([len(S) for S in left[lk]])
        lk += 1
      else:
        H[row, j] = sum([len(S) for S in right[rk]])
        rk += 1
  return(H)

"""
`seq_mults` accepts a list "s" of integers and returns their product "pr".  It 
is used by `formula` below.
"""

def seq_mult(s):
  pr = 1
  for i in range(0, len(s)):
    pr *= s[i]
  return(pr)

In [104]:
"""
We're now ready for the function `formula` which will implement the formula 
in Theorem 5.2.  "m" is the number of vertices in the left branch of the 
main loop of the graph G and "n" is the number of vertices in the right 
branch.  "top" is the list of trees attached to the top vertex in G, "bottom" 
is the list of trees attached to the bottom vertex, "left" is the list of lists 
of trees attached to the left branch vertices, and "right" is the list of lists 
of trees attached to the right branch vertices.  The output "phi" is the 
order number of (G,W,theta).
"""

def formula(m, n, top, bottom, left, right):
  from math import factorial as f
  
  left_trees = []
  for p in range(0, len(left)):
    left_trees += left[p]
    
  right_trees = []
  for q in range(0, len(right)):
    right_trees += right[q]
  
  all_trees = top[0] + bottom[0] + left_trees + right_trees
  
  seqs_01 = all_seqs(m + n, m)
  vert_seqs = all_seqs_2(seqs_01, left, right) 
    
  top_vert = sum(len(TP) for TP in top[0])
     
  w = m + n + 2 + sum([len(T) for T in all_trees])
  
  # `round` is used to fix the floating point precision error incurred:
  phi = round(seq_mult([ord_num_tree(T)/f(len(T)) for T in all_trees]) * \
     sum([f(w - 1) * seq_mult([1/(w - (i + 1) - top_vert - \
                                 sum([seq[j] for j in range(0, i)])) for \
                              i in range(0, m + n + 1)]) for seq in vert_seqs]))
  
  return(phi)

In [66]:
"""
We can use `formula` to verify Example 5.3.
"""

T1 = graph_entry(5)

Enter row-column separated by space. Enter s to stop.2 1
Enter row-column separated by space. Enter s to stop.3 1
Enter row-column separated by space. Enter s to stop.4 3
Enter row-column separated by space. Enter s to stop.5 3
Enter row-column separated by space. Enter s to stop.s


In [54]:
T2 = graph_entry(3)

Enter row-column separated by space. Enter s to stop.2 1
Enter row-column separated by space. Enter s to stop.3 1
Enter row-column separated by space. Enter s to stop.s


In [98]:
formula(2, 1, [[T2]], [[]], [[],[T1]], [[T1, T2]])

2334717665280